In [1]:
import numpy as np

from trimesh import Trimesh, load, graph
from trimesh.exchange.export import export_obj

import meshplot as mp

import open3d as o3d
from copy import deepcopy

import pyvista as pv

from scipy.spatial import distance

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [30]:
def visualize(pc1, pc2):
    o3d.visualization.draw_geometries([pc1, pc2])
    
def copy(pc1, pc2, tr1=None, tr2=None):
    tr1 = (0, 0, 0) if tr1 is None else tr1
    tr2 = (0, 0, 0) if tr2 is None else tr2
    pc1 = deepcopy(pc1).translate(tr1)
    pc2 = deepcopy(pc2).translate(tr2)
    pc1.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (len(pc1.points), 1)))
    pc2.colors = o3d.utility.Vector3dVector(np.tile([0, 0, 1], (len(pc1.points), 1)))
    return pc1, pc2

def double_plot(pc1: o3d.geometry.PointCloud, pc2: o3d.geometry.PointCloud):
    print(pc1)
    pc1, pc2 = copy(pc1, pc2, (-0.05, 0, 0), (0.05, 0, 0))
    visualize(pc1, pc2)

def merged_plot(pc1: o3d.geometry.PointCloud, pc2: o3d.geometry.PointCloud):
    pc1, pc2 = copy(pc1, pc2)
    visualize(pc1, pc2)

In [31]:
BAD_DIST_THRESHOLD = 0.000
CENTER_THRESHOLD = 0.05

Load meshes

In [32]:
ref_pc = o3d.io.read_point_cloud('non_deformed_2_correspondences_zoom_2048.ply')
bad_pc = o3d.io.read_point_cloud('deformed_2_correspondences_zoom_2048.ply')
ref_points = np.asarray(ref_pc.points)
bad_points = np.asarray(bad_pc.points)
double_plot(ref_pc, bad_pc)

PointCloud with 2048 points.
PointCloud with 2048 points.


In [33]:
merged_plot(ref_pc, bad_pc)

PointCloud with 2048 points.
PointCloud with 2048 points.


Find `ref_pc` vertices close to any `bad_pc` vertices

In [34]:
PATCH_SIZE = 500


# Because processing everything in one sweep kills the kernel
def iter_patches(points: np.ndarray):
    for start in range(0, len(points), PATCH_SIZE):
        end = min(start + PATCH_SIZE, len(points))
        yield start, end, points[start:end]


def get_mask_near_mesh(pc1: np.ndarray, pc2: np.ndarray, threshold: float):
    used_vertices = np.zeros(len(ref_pc.points), dtype=bool)
    for start, end, patch1 in iter_patches(pc1):
        for _, _, patch2 in iter_patches(pc2):
            euc_dist=distance.cdist(patch1, patch2)
            used_from_batch = euc_dist.min(axis=1) < threshold
            used_vertices[start:end] = np.bitwise_or(used_vertices[start:end], used_from_batch)
    
    return used_vertices


used_v_mask_bad = get_mask_near_mesh(ref_points, bad_points, BAD_DIST_THRESHOLD)


Remove `ref_pc` vertices which are far from any `bad_pc` vertices

In [35]:
def get_point_cloud_with_mask(src_points, vert_mask):
    points = o3d.utility.Vector3dVector(src_points[vert_mask])
    return o3d.geometry.PointCloud(points)

ref_points = np.asarray(ref_pc.points)
ref_pc_bad = get_point_cloud_with_mask(ref_points, used_v_mask_bad)
double_plot(ref_pc_bad, bad_pc)

PointCloud with 0 points.
PointCloud with 0 points.


Remove vertices which are far from the `bad_pc`'s centroid

In [13]:

def centroid(points):
    points = np.asarray(points)
    return points[points[:, 2].argmax()]


def get_mask_near_point(points, point, threshold):
    used_vertices = np.zeros(len(points), dtype=bool)
    for start, end, patch in iter_patches(points):
        used_from_patch = np.linalg.norm(patch - point, axis=1) < threshold
        used_vertices[start:end] = np.bitwise_or(used_vertices[start:end], used_from_patch)
    
    return used_vertices

c = o3d.geometry.TriangleMesh.create_sphere(radius=0.001).translate(centroid(bad_pc.points))

used_v_mask_centroid = get_mask_near_point(ref_points, centroid(bad_pc.points), CENTER_THRESHOLD)
print(used_v_mask_centroid.sum())

ref_pc_centroid = get_point_cloud_with_mask(ref_points, used_v_mask_centroid)

double_plot(ref_pc_centroid, bad_pc)

648


Combine

In [15]:
used_v_mask_combined = np.bitwise_or(used_v_mask_bad, used_v_mask_centroid)
ref_pc_combined = get_point_cloud_with_mask(ref_points, used_v_mask_combined)
double_plot(ref_pc_combined, bad_pc)

In [16]:
merged_plot(ref_pc_combined, bad_pc)

In [17]:
np.save(f'non_deformed_2_correpondences_zoom_2048_paired_{len(ref_pc_combined.points)}.npy', np.asarray(ref_pc_combined.points))
o3d.io.write_point_cloud(f'non_deformed_2_correspondences_zoom_2048_paired_{len(ref_pc_combined.points)}.ply', ref_pc_combined)

True

In [18]:
np.asarray(ref_pc_combined.points).shape

(648, 3)

In [19]:
bad_pc_combined = get_point_cloud_with_mask(bad_points, used_v_mask_combined)

In [21]:
double_plot(ref_pc_combined, bad_pc_combined)

In [240]:
len(ref_pc_combined.points), len(bad_pc_combined.points)

(648, 648)

In [241]:
np.save(f'deformed_2_correpondences_zoom_2048_paired_{len(bad_pc_combined.points)}.npy', np.asarray(bad_pc_combined.points))
o3d.io.write_point_cloud(f'deformed_2_correspondences_zoom_2048_paired_{len(bad_pc_combined.points)}.ply', bad_pc_combined)

True

In [22]:
ref_1607 = np.asarray(ref_pc_combined.points)
bad_1607 = np.asarray(bad_pc_combined.points)
deformations = bad_1607 - ref_1607
distances = np.linalg.norm(deformations, axis=1)
max_deformation_idx = distances.argmax()
points = [ref_1607[max_deformation_idx], bad_1607[max_deformation_idx]]
line_set = o3d.geometry.LineSet(o3d.utility.Vector3dVector(points), o3d.utility.Vector2iVector([[0, 1]]))
o3d.visualization.draw_geometries([ref_pc_combined, bad_pc_combined, line_set])